# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [2]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.3/872.3 kB 12.5 MB/s eta 0:00:00


In [128]:
from ultralytics import YOLO , solutions
import cv2
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [129]:
model = YOLO('/content/drive/MyDrive/week 7 exam/Dataset/YOLO_Model.pt')

## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [130]:
vid_path = '/content/drive/MyDrive/week 7 exam/Dataset/YOLO_Video.mp4'
#https://docs.ultralytics.com/guides/object-counting/?h=solutions.objectcounter#how-do-i-count-objects-in-a-video-using-ultralytics-yolov8
capture = cv2.VideoCapture(vid_path)

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [131]:
#https://docs.ultralytics.com/guides/object-counting/?h=solutions.objectcounter#how-do-i-count-objects-in-a-video-using-ultralytics-yolov8
height , width , fps = [int(x) for x in (capture.get(cv2.CAP_PROP_FRAME_HEIGHT) , capture.get(cv2.CAP_PROP_FRAME_WIDTH) , capture.get(cv2.CAP_PROP_FPS)) ]

In [132]:
print(f'height: {height}')
print(f'width: {width}')
print(f'fps: {fps}') #725 frames to be processes 29*25

height: 1080
width: 1920
fps: 29


## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [133]:
output_path = '/content/drive/MyDrive/week 7 exam/output.mp4'
#https://docs.ultralytics.com/guides/object-counting/?h=solutions.objectcounter#how-do-i-count-objects-in-a-video-using-ultralytics-yolov8
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path , fourcc , fps , (width , height))

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

In [134]:
#initializing variables for later
left_top_counter = 0
right_top_counter = 0
left_down_counter = 0
right_down_counter = 0
red = (255 , 0 , 0)
green = (0 , 255 , 0)
blue = (0 , 0 , 255)
black = (0,0,0)
x_line = int (height//2)
y_line = int(width//2)
thick = 2
scale = 1
font = cv2.FONT_HERSHEY_SIMPLEX
frame_counter = 0 #just to check for one or two frames

## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [136]:
#https://docs.ultralytics.com/guides/object-counting/?h=solutions.objectcounter#__tabbed_1_1
while capture.isOpened() :
  success , frame = capture.read()
  #frame_counter +=1
  #if frame_counter>3:
    #break
  if not success:
    print('Error, could not open video')
    break
  else:
    results = model(frame)

    for result in results:

      boxes = result.boxes

      for box in boxes:
        #https://docs.ultralytics.com/usage/simple-utilities/#convert-coco-into-yolo-format
        x_min , y_min , x_max , y_max = box.xyxy[0].cpu().numpy()
        conf = box.conf[0].cpu().numpy()
        id = int(box.id[0].cpu().numpy()) if box.id is not None else -1
        clas = int(box.cls[0].cpu().numpy()) if box.cls is not None else -1


        #https://gist.github.com/kodekracker/1777f35e8462b2b21846c14c0677f611
        label = f'ID: {id} Class: {clas} Conf: {conf:.2f}'

        cv2.rectangle(frame , (int(x_min) , int(y_min) - 10) , (int(x_max) , int(y_max)), blue , thick )

        cv2.putText(frame , label , (int(x_min) , int(y_min) - 10)  , font , scale , green , thick)

        #https://stackoverflow.com/questions/49799057/how-to-draw-a-point-in-an-image-using-given-co-ordinate-with-python-opencv
        #https://docs.ultralytics.com/tasks/detect/#why-should-i-use-ultralytics-yolov8-for-object-detection
        cx = x_min+x_max/2
        cy = y_min+y_max/2
        cv2.circle(frame, (int(cx),int(cy)), 5 , black , -1)




        if cx > x_line and cy > y_line:
          left_top_counter+=1
        elif cx < x_line and cy < y_line:
          left_down_counter+=1
        elif cx > x_line and cy > y_line:
          right_top_counter+=1
        else:
          right_down_counter+=1




  #https://docs.opencv.org/4.x/dc/da5/tutorial_py_drawing_functions.html
  cv2.line(frame , (0, x_line) , (width,x_line) , red , thick )

  cv2.line(frame , (y_line, 0) , (y_line,height) , red , thick )

  #https://ai.thestempedia.com/python-function/cv2-puttext/
  cv2.putText(frame , f'Counter: {left_top_counter}' , (10,70) , font , scale , black , thick)
  cv2.putText(frame , f'Counter: {left_down_counter}' , (10,1070) , font , scale , black , thick)

  cv2.putText(frame , f'Counter: {right_top_counter}' , (1300,70) , font , scale , black , thick)
  cv2.putText(frame , f'Counter: {right_down_counter}' , (1300,1070) , font , scale , black , thick)
  cv2.putText(frame , 'Shouq' , (y_line,x_line) , font , scale , black , thick)

  out.write(frame)


0: 384x640 2 buss, 16 cars, 3551.3ms
Speed: 3.5ms preprocess, 3551.3ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 17 cars, 2642.1ms
Speed: 3.9ms preprocess, 2642.1ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 cars, 2263.2ms
Speed: 3.6ms preprocess, 2263.2ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 cars, 2272.9ms
Speed: 3.9ms preprocess, 2272.9ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bus, 17 cars, 2251.8ms
Speed: 3.9ms preprocess, 2251.8ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 cars, 2871.9ms
Speed: 4.2ms preprocess, 2871.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 18 cars, 3289.1ms
Speed: 3.6ms preprocess, 3289.1ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bus, 17 cars, 2276.6ms
Speed: 3.9ms preprocess, 2276.6ms inference, 

In [137]:
#https://docs.ultralytics.com/guides/object-counting/?h=solutions.objectcounter#how-do-i-count-objects-in-a-video-using-ultralytics-yolov8
capture.release()
out.release()
cv2.destroyAllWindows()

## Save and Submit Your Work

Submit both the notebook and the output video